In [3]:
import os
import numpy as np
import tensorflow as tf
import keras
from keras import layers
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
dataset, info = tfds.load('oxford_iiit_pet:3.2.0', with_info=True)

def normalize(input_image, input_mask):
    input_image = tf.cast(input_image, tf.float32) / 255.0
    input_mask -= 1
    return input_image, input_mask

def load_image_train(datapoint):
    input_image = tf.image.resize(datapoint['image'], (128, 128))
    input_mask = tf.image.resize(datapoint['segmentaiton_mask'], (128,128))
    if tf.random.uniform(()) > 0.5:
        input_image = tf.image.flip_left_right(input_image)
        input_mask = tf.image.flip_left_right(input_mask)
    input_image, input_mask = normalize(input_image, input_mask)
    return input_image, input_mask

def load_image_test(datapoint):
    input_image = tf.image.resize(datapoint['image'], (128,128))
    input_mask = tf.image.resize(datapoint['segmentation_mask'], (128,128))
    input_image, input_mask = normalize(input_image, input_mask)
    return input_image, input_mask

TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000

STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE
train = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test = dataset['test'].map(load_image_test)

train_dataset = trian.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test.batch(BATCH_SIZE)

def build_unet(input_shape):
    inputs = keras.Input(shape=input_shape)
    conv1 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2,2))(conv1)
    
    conv2 = layers.Conv2D(128, (3,3), activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, (3,3), activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size(2,2))(conv2)
    
    conv3 = layers.Conv2D(256, (3,3), activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, (3,3), activation='relu', padding='same')(conv3)
    
    up1 = layers.concatenate([layers.UpSampling2D(size=(2,2))(conv3), conv2], axis=-1)
    conv4 = layers.Conv2D(128, (3,3), activation='relu', padding='same')(up1)
    conv4 = layers.Conv2D(128, (3,3), activation='relu', padding='same')(conv4)
    
    up2 = layers.concatenate([layers.UpSampling2D(size=(2,2))(conv4), conv1], axis=-1)
    conv5 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(up2)
    conv5 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(conv5)
    
    outputs = layers.Conv2D(3,(1,1), activation='softmax')(conv5)
    return keras.Model(inputs=inputs, outputs=outputs)

model = build_unet((128, 128, 3))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
EPOCHS = 10
VAL_SUBSPLITS = 5
VALIDATION_STEPS = info.splits['test'].num_examples//BATCH_SIZE//VAL_SUBSPLITS

model_history = model.fit(train_dataset, epochs=EPOCHS,
                         steps_per_epoch = STEPS_PER_EPOCH,
                         validation_steps = VALIDATION_STEPS,
                         validation_data = test_dataset)

In [ ]:
for image, mask in test_dataset.take(1):
    pred_mask = model.predict(image)
    plt.figure(figsize=(10,10))
    for i in range(9):
        plt.subplot(3,3,i+1)
        plt.imshow(image[i])
        plt.imshow(np.argmax(pred_max[i], axis=-1), alpha=0.5)
        plt.axis("off")
    plt.show